# **Precio promedio - Recámaras**

In [1]:
# Manejo de datos
import os # Directorios
import pandas as pd # Manipulación df
# Gráficas
import plotly.graph_objects as go #Para obtener librería usar: pip install plotly
from plotly.subplots import make_subplots
import plotly.io as pio # Exportar gráfica

# Obtener el directorio actual de trabajo
directorio_actual = os.getcwd()

# Directorio donde se encuentran los archivos JSON (ruta relativa)
directorio_json = os.path.join(directorio_actual, '../../db/datos_json')

# Obtener la lista de archivos JSON en el directorio
archivos_json = os.listdir(directorio_json)

# Cargar los archivos JSON y crear DataFrames
for archivo in archivos_json:
    nombre_tabla = archivo.replace('datos_', '').replace('.json', '')
    ruta_json = os.path.join(directorio_json, archivo)
    globals()[f"df_{nombre_tabla}"] = pd.read_json(ruta_json)

# Obtener todos los nombres de las variables globales
nombres_variables_globales = list(globals().keys())

# Filtrar los nombres que comienzan con "df_", contienen "alfa_q" y "pachuca"
nombres_df_filtrados = [
    nombre for nombre in nombres_variables_globales 
    # Caso de cuando no son las alfa q
    if nombre.startswith("df_") and "alfa" in nombre and "puebla" in nombre and "2024" in nombre

    #if nombre.startswith("df_") and "pachuca" in nombre    
    #and ("alfa_q" in nombre or "jul_2023" in nombre or "sep_2023" in nombre or "feb_2024" in nombre or "mar_2024" in nombre or "may_2024" in nombre)
]

# Imprimir la lista de DataFrames filtrados
print("Lista de DataFrames filtrados:")
nombres_df_filtrados

Lista de DataFrames filtrados:


['df_alfa_agosto_2024_puebla',
 'df_alfa_febrero_2024_puebla',
 'df_alfa_julio_2024_puebla',
 'df_alfa_junio_2024_puebla',
 'df_alfa_septiembre_2024_puebla']

In [2]:
# Iterar sobre cada DataFrame en la lista filtrada
for nombre_df in nombres_df_filtrados:
    # Obtener el DataFrame usando globals()
    df = globals()[nombre_df]
    
    df.rename(columns={'Preciol_23':'precio','Baño_total': 'baño_total','Banos_Total':'baño_total','Precio':'precio','recamaras':'Recamaras'}, inplace=True)
        # Asignar el DataFrame modificado de nuevo a la variable global
    globals()[nombre_df] = df 

# Imprimir confirmación
print("Columnas renombradas en los DataFrames filtrados.")


Columnas renombradas en los DataFrames filtrados.


In [3]:
# Crear una lista de DataFrames seleccionados con las columnas específicas
dataframes_list = []
for nombre_df in nombres_df_filtrados:
    # Seleccionar las columnas 'id' y 'categoria'
    segment_df = globals()[nombre_df][['id', 'Categoria', 'precio', 'propiedad', 'metros_total',
       'metros_construido', 'precio_m2_terreno', 'precio_m2_construido',
       'tiempo_de_publicacion', 'Estado', 'Tipo', 'Estacionamientos',
       'Recamaras', 'Banos', 'Medio_banos', 'baño_total', 'Seguridad',
       'Colonia', 'CP', 'url']]
    # Añadir el DataFrame a la lista
    dataframes_list.append(segment_df)
# Concatenar todos los DataFrames individuales en uno solo
df_concatenado = pd.concat(dataframes_list)

## Eliminación de duplicados

In [4]:
# Eliminar por duplicado general, osea los registros exactamente iguales
print(df_concatenado[df_concatenado.duplicated()])

   id Categoria    precio                                    propiedad  \
1   2        E1  280000.0  Venta de Departamento en Puebla de zaragoza   

   metros_total  metros_construido  precio_m2_terreno  precio_m2_construido  \
1            70                 70             4000.0                4000.0   

   tiempo_de_publicacion Estado  Tipo  Estacionamientos  Recamaras  Banos  \
1                      0  Venta  Casa                 1          2      1   

   Medio_banos  baño_total  Seguridad               Colonia CP  \
1            0         1.0          0  Las Hadas Mundial 86  0   

                                                 url  
1  https://www.lamudi.com.mx/detalle/41032-73-b1c...  


In [5]:
df_concatenado.drop_duplicates(inplace=True)

In [6]:
# Agrupar por los registros duplicados y contar las ocurrencias
agrupados = df_concatenado[df_concatenado.duplicated(subset=['metros_construido','Colonia', 'CP', 'precio','Estacionamientos','Recamaras','propiedad'], keep=False)].groupby(['metros_construido','Colonia', 'CP', 'precio','Recamaras','propiedad']).size().reset_index(name='count')
print(agrupados)

      metros_construido                Colonia CP       precio  Recamaras  \
0                    30     San Miguel La Rosa  0     850000.0          1   
1                    34                 La Paz  0    1333000.0          1   
2                    35  Guadalupe Xonacatepec  0    1062799.0          2   
3                    38                Alpha 2  0    1668744.0          1   
4                    40   Rincon de San Andres  0    1350000.0          2   
...                 ...                    ... ..          ...        ...   
3956              37594              La Calera  0    5490000.0          3   
3957              39525                Anzures  0    7900000.0          3   
3958              40642           Pueblo Nuevo  0    1800000.0          2   
3959              85571       Centro Historico  0    8000000.0          6   
3960             220000       Centro Historico  0  335410000.0         20   

                                              propiedad  count  
0         

In [7]:
# Eliminar duplicados, conservando la primera aparición
df_sin_duplicados = df_concatenado.drop_duplicates(subset=['metros_construido','Colonia', 'CP', 'precio','Estacionamientos','Recamaras','baño_total','propiedad'], keep='first')
# conservar la última aparición
# df_sin_duplicados = df_concatenado.drop_duplicates(keep='last')
print(df_sin_duplicados)

        id Categoria       precio  \
0        1        E1     280000.0   
1        2        E1     280000.0   
2        3        E1     315000.0   
3        4        E1     323656.0   
4        5        E1     324900.0   
...    ...       ...          ...   
3531  3532        L+   21000000.0   
3532  3533        L+   21000000.0   
3533  3534        L+   21000000.0   
3558  3559        EL   28783500.0   
3581  3582        EL  149922400.0   

                                              propiedad  metros_total  \
0                              SE VENDE CASA TIM 662659            47   
1           Venta de Departamento en Puebla de zaragoza            70   
2     VENTA DE CASA EN PLAZAS AMALUCAN, HEROÍCA PUEB...           151   
3              HERMOSA CASA EN LOS HEROES DE PUEBLA !!!            70   
4     GRAN OPORTUNIDAD, Hermosa Casa en Plazas Amalu...           150   
...                                                 ...           ...   
3531  Departamento en VENTA en PueblaTorre I

## Diidir por tipo

In [8]:
df_sin_duplicados['Tipo'].unique()

array(['Casa', 'Departamento'], dtype=object)

In [9]:
casas = df_sin_duplicados[df_sin_duplicados['Tipo'].isin(['Casa', 'Casa En Fraccionamiento', 'Casa En Condominio'])]
depas = df_sin_duplicados[df_sin_duplicados['Tipo'].isin(['Departamento', 'Condominio Horizontal'])]

# Resultados

## Casas

In [10]:
casas['Recamaras'] = casas['Recamaras'].astype(int)
casas = casas[(casas['Recamaras'] >= 1) & (casas['Recamaras'] <= 8)]
casas['Recamaras'].unique()

C:\Users\yoe11\AppData\Local\Temp\ipykernel_6728\1015311021.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  casas['Recamaras'] = casas['Recamaras'].astype(int)


array([2, 3, 4, 5, 6, 1, 7, 8])

In [11]:
# Ajustar los valores para que no superen un #
casas.loc[:, 'Recamaras'] = casas['Recamaras'].clip(lower=1,upper=8)
# Calcular el promedio de precio por el número único 
df_promedio_casas = casas.groupby('Recamaras')['precio'].mean().reset_index()
df_promedio_casas

,Recamaras,precio
0,1,3.785650e+06
1,2,1.830492e+06
2,3,3.431574e+06
3,4,4.808423e+06
4,5,6.087080e+06
5,6,9.192189e+06
6,7,8.775000e+06
7,8,9.398636e+06


In [12]:
import pandas as pd
import plotly.graph_objects as go
import locale

# Establecer configuración regional para el formato de precios
locale.setlocale(locale.LC_ALL, '')

# Definir función para formatear precios
def format_price(price):
    return locale.currency(price, grouping=True)

# Crear una lista de etiquetas para el eje x
etiquetas = [str(recamaras) if recamaras != 16 else '16+' for recamaras in df_promedio_casas['Recamaras']]

# Obtener los precios y el número de recámaras
precios = df_promedio_casas['precio']

# Crear el texto con los precios formateados
text_precios = [format_price(precio) for precio in precios]

colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6','#4225CF','#2CBA4D','#2962ff', '#9500ff', '#ff0059']

fig = go.Figure()
fig.add_trace(go.Bar(
    x=etiquetas,
    y=precios,
    text=text_precios,
    marker_color=colores,
    textposition='auto' 
))
fig.update_layout(
    #title='Precio promedio por número de baños',
    xaxis=dict(title='Recámaras'),
    yaxis=dict(title='$', gridcolor='#dddcda', ),
    bargap=0.1,
    bargroupgap=0.05,
    plot_bgcolor='rgba(0,0,0,0)',
    margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)        
)

fig.show()

In [13]:
recam_five = casas[casas['Recamaras']==1]
recam_five[['precio','Recamaras','Tipo']]

,precio,Recamaras,Tipo
476,650000.0,1,Casa
793,790000.0,1,Casa
1369,1490000.0,1,Casa
1529,1848000.0,1,Casa
2062,2927639.0,1,Casa
2162,3200000.0,1,Casa
2530,4100000.0,1,Casa
2632,4500000.0,1,Casa
2636,4500000.0,1,Casa
2637,4500000.0,1,Casa


In [14]:
casas = pd.concat([casas[casas['Recamaras'] != 1], casas[casas['Recamaras'] == 1].drop_duplicates(subset=['precio', 'Recamaras', 'Tipo'])])
recam_five = casas[casas['Recamaras']==1]
recam_five[['precio','Recamaras']].sort_values(by='precio')

,precio,Recamaras
107,495000.0,1
199,570000.0,1
476,650000.0,1
427,700000.0,1
793,790000.0,1
748,885480.0,1
914,1090000.0,1
980,1196000.0,1
1369,1490000.0,1
1271,1500000.0,1


In [15]:
registro_especifico = casas[casas['precio']==18000000]
registro_especifico[['precio','Recamaras']]

,precio,Recamaras
3385,18000000.0,2
3388,18000000.0,3
3645,18000000.0,2
3646,18000000.0,4
3648,18000000.0,3
3649,18000000.0,4
3652,18000000.0,4
3998,18000000.0,2
4000,18000000.0,4
4002,18000000.0,4


In [16]:
registro_especifico = casas[casas['precio']==5500000]
registro_especifico[['precio','Recamaras']]

,precio,Recamaras
2822,5500000.0,3
2823,5500000.0,4
2825,5500000.0,3
2826,5500000.0,3
2827,5500000.0,3
2828,5500000.0,4
2829,5500000.0,3
2830,5500000.0,3
2831,5500000.0,4
2832,5500000.0,4


In [17]:
registro_especifico = casas[casas['precio']==5200000]
registro_especifico[['precio','Recamaras']]

,precio,Recamaras
2773,5200000.0,3
2774,5200000.0,4
2775,5200000.0,3
2776,5200000.0,3
2777,5200000.0,3
3130,5200000.0,3
3360,5200000.0,3
3361,5200000.0,3
3362,5200000.0,4
3363,5200000.0,3


In [18]:
registro_especifico = casas[casas['precio']==4500000]
registro_especifico[['precio','Recamaras']]

,precio,Recamaras
2621,4500000.0,5
2623,4500000.0,3
2624,4500000.0,3
2625,4500000.0,3
2626,4500000.0,5
2627,4500000.0,4
2628,4500000.0,3
2630,4500000.0,4
2631,4500000.0,4
2633,4500000.0,5


In [19]:
casas = casas[~((casas['precio']==18000000) & (casas['Recamaras']==1))]
casas = casas[~((casas['precio']==5500000) & (casas['Recamaras']==1))]
casas = casas[~((casas['precio']==5200000) & (casas['Recamaras']==1))]
casas = casas[~((casas['precio']==4500000) & (casas['Recamaras']==1))]

recam_five = casas[casas['Recamaras']==1].sort_values(by='precio')
recam_five[['precio', 'Recamaras', 'Tipo']]

,precio,Recamaras,Tipo
107,495000.0,1,Casa
199,570000.0,1,Casa
476,650000.0,1,Casa
427,700000.0,1,Casa
793,790000.0,1,Casa
748,885480.0,1,Casa
914,1090000.0,1,Casa
980,1196000.0,1,Casa
1369,1490000.0,1,Casa
1271,1500000.0,1,Casa


In [20]:
casas = pd.concat([casas[casas['Recamaras'] != 6], casas[casas['Recamaras'] == 6].drop_duplicates(subset=['precio', 'Recamaras', 'Tipo'])])
recam_five = casas[casas['Recamaras']==6]
recam_five[['precio','Recamaras']].sort_values(by='precio')

,precio,Recamaras
56,422000.0,6
95,480000.0,6
145,507000.0,6
225,590000.0,6
412,624000.0,6
...,...,...
3441,27500000.0,6
3445,29000000.0,6
3463,38000000.0,6
3471,55000000.0,6


In [23]:
registro_especifico = casas[casas['precio']==422000]
registro_especifico[['precio','Recamaras']]

,precio,Recamaras
53,422000.0,3
54,422000.0,3
55,422000.0,3
57,422000.0,3
58,422000.0,3
56,422000.0,6


In [24]:
registro_especifico = casas[casas['precio']==480000]
registro_especifico[['precio','Recamaras']]

,precio,Recamaras
110,480000.0,3
111,480000.0,3
112,480000.0,3
94,480000.0,2
95,480000.0,6


In [26]:
registro_especifico = casas[casas['precio']==624000]
registro_especifico[['precio','Recamaras']]

,precio,Recamaras
402,624000.0,3
403,624000.0,4
404,624000.0,4
405,624000.0,4
406,624000.0,4
407,624000.0,4
408,624000.0,2
409,624000.0,2
410,624000.0,3
411,624000.0,4


In [29]:
registro_especifico = casas[casas['precio']==60000000]
registro_especifico[['precio','Recamaras']]

,precio,Recamaras
3473,60000000.0,3
3711,60000000.0,4
4078,60000000.0,3
3472,60000000.0,6


In [45]:
casas = casas[~((casas['precio']==422000) & (casas['Recamaras']==6))]
casas = casas[~((casas['precio']==480000) & (casas['Recamaras']==6))]
casas = casas[~((casas['precio']==624000) & (casas['Recamaras']==6))]
casas = casas[~((casas['precio']==60000000) & (casas['Recamaras']==6))]
casas = casas[~((casas['precio']==55000000) & (casas['Recamaras']==6))]
casas = casas[~((casas['precio']==38000000) & (casas['Recamaras']==6))]

recam_five = casas[casas['Recamaras']==6].sort_values(by='precio')
recam_five[['precio', 'Recamaras', 'Tipo']]

,precio,Recamaras,Tipo
145,507000.0,6,Casa
225,590000.0,6,Casa
335,648798.0,6,Casa
1116,1124000.0,6,Casa
1065,1280000.0,6,Casa
1406,1680000.0,6,Casa
1908,2600000.0,6,Casa
2057,2650000.0,6,Casa
2233,2785000.0,6,Casa
2101,3000000.0,6,Casa


In [46]:
casasss = casas.copy()

In [48]:
promedios = casasss.groupby('Recamaras')['precio'].mean().reset_index()
promedios

,Recamaras,precio
0,1,1.658257e+06
1,2,1.830492e+06
2,3,3.431574e+06
3,4,4.808423e+06
4,5,6.087080e+06
5,6,7.713118e+06
6,7,8.775000e+06
7,8,9.398636e+06


In [50]:
import pandas as pd
import plotly.graph_objects as go
import locale

# Establecer configuración regional para el formato de precios
locale.setlocale(locale.LC_ALL, '')

# Definir función para formatear precios
def format_price(price):
    return locale.currency(price, grouping=True)

# Crear una lista de etiquetas para el eje x
etiquetas = [str(recamaras) if recamaras != 16 else '16+' for recamaras in promedios['Recamaras']]

# Obtener los precios y el número de recámaras
precios = promedios['precio']

# Crear el texto con los precios formateados
text_precios = [format_price(precio) for precio in precios]

colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6','#4225CF','#2CBA4D','#2962ff', '#9500ff', '#ff0059']

fig = go.Figure()
fig.add_trace(go.Bar(
    x=etiquetas,
    y=precios,
    text=text_precios,
    marker_color=colores,
    textposition='auto' 
))
fig.update_layout(
    #title='Precio promedio por número de baños',
    xaxis=dict(title='Recámaras'),
    yaxis=dict(title='$', gridcolor='#dddcda', ),
    bargap=0.1,
    bargroupgap=0.05,
    plot_bgcolor='rgba(0,0,0,0)',
    margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)        
)

fig.show()

In [51]:
# Función para guardar la gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    import os
    import plotly.io as pio

    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)

    # Configuración personalizada para la gráfica
    config = {
        'displaylogo': False,  # Ocultar el logo de Plotly
        'modeBarButtonsToRemove': [
            'toImage',       # Botón para guardar como imagen
            'select2d',      # Box select
            'lasso2d',       # Lasso select
            'resetScale2d',  # Reset Axes
        ]
    }

    # Guardar la gráfica como archivo HTML
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html', config=config)

# Ejemplo de uso
guardar_grafico_como_html(fig, 'g_bar_precio_promedio_recamaras_casa',  carpeta='assets/graficas')


## Depa

In [52]:
depas['Recamaras'] = depas['Recamaras'].astype(int)
depas = depas[(depas['Recamaras'] >= 1) & (depas['Recamaras'] <= 8)]
depas['Recamaras'].unique()

C:\Users\yoe11\AppData\Local\Temp\ipykernel_6728\1542590914.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



array([2, 3, 1, 4, 6])

In [53]:
# Ajustar los valores para que no superen un #
depas.loc[:, 'Recamaras'] = depas['Recamaras'].clip(lower=1,upper=8)
# Calcular el promedio de precio por el número único 
df_promedio_depas = depas.groupby('Recamaras')['precio'].mean().reset_index()
df_promedio_depas

,Recamaras,precio
0,1,2.748649e+06
1,2,2.652657e+06
2,3,3.931465e+06
3,4,8.773572e+06
4,6,1.271250e+07


In [54]:
import pandas as pd
import plotly.graph_objects as go
import locale

# Establecer configuración regional para el formato de precios
locale.setlocale(locale.LC_ALL, '')

# Definir función para formatear precios
def format_price(price):
    return locale.currency(price, grouping=True)

# Crear una lista de etiquetas para el eje x
etiquetas = [str(recamaras) if recamaras != 16 else '16+' for recamaras in df_promedio_depas['Recamaras']]

# Obtener los precios y el número de recámaras
precios = df_promedio_depas['precio']

# Crear el texto con los precios formateados
text_precios = [format_price(precio) for precio in precios]

colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6','#4225CF','#2CBA4D','#2962ff', '#9500ff', '#ff0059']

fig = go.Figure()
fig.add_trace(go.Bar(
    x=etiquetas,
    y=precios,
    text=text_precios,
    marker_color=colores,
    textposition='auto' 
))
fig.update_layout(
    #title='Precio promedio por número de baños',
    xaxis=dict(title='Recámaras'),
    yaxis=dict(title='$', gridcolor='#dddcda', ),
    bargap=0.1,
    bargroupgap=0.05,
    plot_bgcolor='rgba(0,0,0,0)',
    margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)        
)

fig.show()

In [55]:
recam_five = depas[depas['Recamaras']==1].sort_values(by='precio')
recam_five[['precio', 'Recamaras', 'Tipo']]

,precio,Recamaras,Tipo
361,650000.0,1,Departamento
1013,850000.0,1,Departamento
698,850000.0,1,Departamento
680,865000.0,1,Departamento
682,865000.0,1,Departamento
...,...,...,...
3305,6400000.0,1,Departamento
3459,9000000.0,1,Departamento
4308,9000000.0,1,Departamento
3722,9000000.0,1,Departamento


In [56]:
depas = pd.concat([depas[depas['Recamaras'] != 1], depas[depas['Recamaras'] == 1].drop_duplicates(subset=['precio', 'Recamaras', 'Tipo'])])
recam_five = depas[depas['Recamaras']==1].sort_values(by='precio')
recam_five[['precio', 'Recamaras', 'Tipo']]

,precio,Recamaras,Tipo
361,650000.0,1,Departamento
1013,850000.0,1,Departamento
680,865000.0,1,Departamento
875,990000.0,1,Departamento
1281,1115000.0,1,Departamento
...,...,...,...
3133,5249035.0,1,Departamento
4006,5778864.0,1,Departamento
3276,6115945.0,1,Departamento
3305,6400000.0,1,Departamento


In [70]:
registro_especifico = depas[depas['precio']==6115945]
registro_especifico[['precio','Recamaras']]

,precio,Recamaras
4006,5778864.0,1


In [67]:
depas = depas[~((depas['precio']==9000000) & (depas['Recamaras']==1))]
depas = depas[~((depas['precio']==6400000) & (depas['Recamaras']==1))]
depas = depas[~((depas['precio']==6115945) & (depas['Recamaras']==1))]


recam_five = depas[depas['Recamaras']==1].sort_values(by='precio')
recam_five[['precio', 'Recamaras', 'Tipo']]

,precio,Recamaras,Tipo
361,650000.0,1,Departamento
1013,850000.0,1,Departamento
680,865000.0,1,Departamento
875,990000.0,1,Departamento
1281,1115000.0,1,Departamento
...,...,...,...
3742,4600000.0,1,Departamento
3231,4664000.0,1,Departamento
3031,4664480.0,1,Departamento
3133,5249035.0,1,Departamento


In [68]:
# Ajustar los valores para que no superen un #
depas.loc[:, 'Recamaras'] = depas['Recamaras'].clip(lower=1,upper=8)
# Calcular el promedio de precio por el número único 
df_promedio_depas = depas.groupby('Recamaras')['precio'].mean().reset_index()
df_promedio_depas

,Recamaras,precio
0,1,2.606990e+06
1,2,2.652657e+06
2,3,3.931465e+06
3,4,8.773572e+06
4,6,1.271250e+07


In [69]:
import pandas as pd
import plotly.graph_objects as go
import locale

# Establecer configuración regional para el formato de precios
locale.setlocale(locale.LC_ALL, '')

# Definir función para formatear precios
def format_price(price):
    return locale.currency(price, grouping=True)

# Crear una lista de etiquetas para el eje x
etiquetas = [str(recamaras) if recamaras != 16 else '16+' for recamaras in df_promedio_depas['Recamaras']]

# Obtener los precios y el número de recámaras
precios = df_promedio_depas['precio']

# Crear el texto con los precios formateados
text_precios = [format_price(precio) for precio in precios]

colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6','#4225CF','#2CBA4D','#2962ff', '#9500ff', '#ff0059']

fig = go.Figure()
fig.add_trace(go.Bar(
    x=etiquetas,
    y=precios,
    text=text_precios,
    marker_color=colores,
    textposition='auto' 
))
fig.update_layout(
    #title='Precio promedio por número de baños',
    xaxis=dict(title='Recámaras'),
    yaxis=dict(title='$', gridcolor='#dddcda', ),
    bargap=0.1,
    bargroupgap=0.05,
    plot_bgcolor='rgba(0,0,0,0)',
    margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)        
)

fig.show()

In [71]:
# Función para guardar la gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    import os
    import plotly.io as pio

    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)

    # Configuración personalizada para la gráfica
    config = {
        'displaylogo': False,  # Ocultar el logo de Plotly
        'modeBarButtonsToRemove': [
            'toImage',       # Botón para guardar como imagen
            'select2d',      # Box select
            'lasso2d',       # Lasso select
            'resetScale2d',  # Reset Axes
        ]
    }

    # Guardar la gráfica como archivo HTML
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html', config=config)

# Ejemplo de uso
guardar_grafico_como_html(fig, 'g_bar_precio_promedio_recamaras_depa',  carpeta='assets/graficas')
